In [1]:
import urllib.request

def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2023):
    with open('sessionID') as f:
        sessionId = f.readlines()[0].strip()
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data
    
import re
def find_int_in_string(val):
    return list(map(int, re.findall('\\d+', val)))
    
def convert(three_vals, val):
    out = val
    destination, source, delta = three_vals
    if source <= val < (source + delta):
        out = val + destination - source
    return out

In [2]:
import itertools
from collections import defaultdict

seeds, *map_list = [list(y) for x, y in itertools.groupby(get_data(5), lambda z: z == '') if not x]

out = []
seed_list = find_int_in_string(seeds[0])
out.append(seed_list)
for ii, mapping in enumerate(map_list):
    mapped = []
    for val in out[ii]:
        for line in mapping[1:]:
            new = convert(find_int_in_string(line), val)
            if new != val:
                break
        mapped.append(new)
    out.append(mapped)
print(min(out[-1]))

324724204


cases:
ss----------se
                  ms-----------me  

keep [ss, se] only pre

ss----------se
      ms-----------me  

keep: [ss, ms), map [ms, se] pre and overlap

ss-------------------se
    ms-----------me    

keep: [ss, ms), map [ms, me], keep: (me, se) all three

       ss----------se
    ms------------------me    

map: [ss, se) only overlap


            ss-------------------se
    ms-----------me    

keep: map [ss, me], keep: (me, se) overlap, and post


                        ss-------------------se
    ms-----------me  
keep: map [ss, se] only post

three sections: pre, overlap, post

In [ ]:
pairs = list(zip(seed_list[::2], seed_list[1::2]))
samples = [(val[0], val[0]+val[1]) for val in pairs]
part2 = []
for pair in samples:
    mapped = [pair]
    for mapping in map_list:
        new_mapped = []
        for line in mapping[1:]:
            new_unmapped = []
            while mapped:
                d,s,r = find_int_in_string(line)

                ss, se = mapped.pop()
                ms = s
                me = s + r
                
                pre = (ss,min(se,ms))
                overlap = (max(ss, ms), min(me, se))
                post = (max(me, ss), se)
                
                # add pre section 
                if pre[1] > pre[0]:
                    new_unmapped.append(pre)
                if overlap[1] > overlap[0]:
                    new_mapped.append((overlap[0]-ms+d, overlap[1]-ms+d))
                if post[1] > post[0]:
                    new_unmapped.append(post)
            mapped = new_unmapped
        mapped = new_mapped + new_unmapped
    part2.append(min(mapped)[0])
print(min(part2))
    

3213531697
